<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [ ]:
def long_form(df_plot):
    return df_plot.melt('ds', var_name='unique_id', value_name='price')

def altair_plot(df_plot): 
    import altair as alt
    highlight = alt.selection_point(on='mouseover', fields=['unique_id'], bind='legend', nearest=True)

    base = alt.Chart(df_plot).encode(
        x='ds:T',
        y='price:Q',
        color='unique_id:N'
    )

    points = base.mark_circle().encode(
        opacity=alt.value(0)
    ).add_params(
        highlight
    ).properties(
        width=1000
    )

    lines = base.mark_line().encode(
        size=alt.condition(~highlight, alt.value(1), alt.value(3))
    )

    return points + lines

In [ ]:
url1 = 'http://api.weatherapi.com/v1/current.json?key=e7e650398d6447f7a35121707230412&q=Brisbane&aqi=yes'
url2 = 'http://api.weatherapi.com/v1/forecast.json?key=e7e650398d6447f7a35121707230412&q=Brisbane&days=10&aqi=no&alerts=no'
url3 = 'http://api.weatherapi.com/v1/history.json?key=e7e650398d6447f7a35121707230412&q=Brisbane&dt=2023-01-01'

In [ ]:
class Weather_api_url:
    def __init__(self, token:str= None, dataType:str = 'json', location:str = 'Brisbane', api:str = 'no'):
        self.url = 'http://api.weatherapi.com/v1/'
        self.token = token
        self.dataType = dataType
        self.location = location
        self.aqi = api
        
    def __str__(self):
        return self.url + '<detail_type_ex.current><json,xml>?key=<token>&q=<location>&aqi=<yes,no>'
        
    def current(self):
        url = self.url + 'current.' + self.dataType+'?key='+ self.token + '&q=' + self.location + '&aqi=' + self.aqi
        return url

    
    def forecast(self, days:int = 10):
        url = self.url + 'forecast.' + self.dataType+'?key='+ self.token + '&q=' + self.location + '&days=' + str(days) + '&aqi=' + self.aqi + '&alerts=no'
        return url
    
    def history(self, date:str = '2023-01-01'):
        url = self.url + 'history.' + self.dataType+'?key='+ self.token + '&q=' + self.location + '&dt=' + date
        return url

In [ ]:
from io import StringIO

def weather_request(url):
    import requests
    import json
    # Make a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()
        # Print or work with each array
        return data
    else:
        # Print an error message if the request was not successful
        print(f"Error: {response.status_code}")

        
def data_filter(data, filter_sub: str = 'forecast'):
    import json
    import pandas as pd
    
    df = pd.DataFrame()
    location = data.get("location")
    current = data.get("current")
    forecast = data.get("forecast")
    
    if filter_sub == 'forecast':
        if forecast is None:
            print("The variable is None")
            return
        else:
            forecastday = forecast.get("forecastday")
            
            for days in forecastday:
                a = days.get("hour", [])
                for rows in a:
                    new = pd.DataFrame(rows)
                    new.drop(['time_epoch', 'condition', 'wind_dir', 'pressure_in', 'wind_mph', 'precip_in', 
                            'temp_f', 'heatindex_f', 'feelslike_f', 'windchill_f', 'dewpoint_f', 'will_it_snow',
                            'will_it_rain', 'chance_of_snow', 'vis_miles','gust_mph' ], axis=1, inplace=True)
                    df = pd.concat([df, new[0:1]], ignore_index=True)
                    
            
            df.rename(columns={'time': 'ds'}, inplace=True)
                    
    elif filter_sub == 'current':
        if current is None:
            print("The variable is None")
            return
        else:
            df = pd.DataFrame(current)
            df.drop(['last_updated_epoch', 'condition', 'wind_dir', 'pressure_in', 'wind_mph', 'precip_in', 
                             'temp_f', 'feelslike_f', 'vis_miles','gust_mph' ], axis=1, inplace=True)
            df.rename(columns={'last_updated': 'ds'}, inplace=True)
        
    else:
        if location is None:
            print("The variable is None")
            return
        else:
            df = pd.DataFrame(location, index=[0]) 

    return df

In [ ]:
def data_scaler(df):
    import pandas as pd
    from sklearn.preprocessing import MinMaxScaler

    numeric_columns = df.select_dtypes(include=['number']).columns
        
    # Scale numeric columns
    df[numeric_columns] = MinMaxScaler().fit_transform(df[numeric_columns])


    return df

In [ ]:
def his_data(token, date):
    url = Weather_api_url(token).history(date) #creates the link to retrieve json data
    data_json = weather_request(url) #returns the json data
    
    df = data_filter(data_json, 'forecast')  #turns json into pd and filters the data
    
    return df    #return pd dateframe

In [ ]:
def historical_data(token, start_date = '2023-01-01', end_date = '2023-01-10', freq = 'D', path = None, name = 'weather'):
    from tqdm.notebook import trange, tqdm
    import pandas as pd
    
    rng = pd.date_range(start_date, end_date, freq=freq) #create date list between start and end date

    dates =  rng.strftime('%Y-%m-%d')  #change date format from timestamp to date

    df = pd.DataFrame()
    for date in tqdm(dates, desc="Downloading Days", unit="days"):    #loop through date in date list
        new = his_data(token, date)      #download pd dataframe for the day
        df = pd.concat([df, new], ignore_index=True)  #add the pd dataframe to main pd dataframe 
    

    if path is None:
        pass
    else:
        df.to_csv(f'{path}/{name}.csv', index=False) #save the data as a csv
    
    return pd

In [ ]:
startDate= '2023-01-01'
endDate = '2023-12-10'
freq = 'D'
path = 'Data'
name = 'weather'

```python

historical_data(token,
                startDate,
                endDate,
                freq,
                path,
                name
                )
```

In [ ]:
import pandas as pd
df = pd.read_csv(f'{path}/{name}.csv')
df.shape

(8256, 17)

In [ ]:
df[-24:]

,ds,temp_c,is_day,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feelslike_c,windchill_c,heatindex_c,dewpoint_c,chance_of_rain,vis_km,gust_kph,uv
8232,2023-12-10 00:00,22.3,0,9.0,59,1017.0,0.01,77,88,24.7,22.3,24.7,18.1,0,10.0,14.9,1.0
8233,2023-12-10 01:00,22.9,0,9.7,67,1016.0,0.00,72,86,25.0,22.9,25.0,17.7,0,10.0,15.2,1.0
8234,2023-12-10 02:00,23.8,0,11.9,72,1016.0,0.00,66,68,25.5,23.8,25.5,17.0,0,10.0,17.0,1.0
8235,2023-12-10 03:00,24.0,0,13.3,74,1015.0,0.00,64,64,25.5,24.0,25.5,16.7,0,10.0,18.8,1.0
8236,2023-12-10 04:00,24.0,0,14.0,75,1015.0,0.00,64,68,25.5,24.0,25.5,16.6,0,10.0,20.0,1.0
8237,2023-12-10 05:00,24.1,1,15.5,76,1016.0,0.00,63,59,25.6,24.1,25.6,16.6,0,10.0,21.7,6.0
8238,2023-12-10 06:00,24.4,1,16.2,83,1016.0,0.00,62,61,25.8,24.4,25.8,16.5,0,10.0,21.8,6.0
8239,2023-12-10 07:00,24.7,1,16.6,84,1017.0,0.01,61,69,26.0,24.7,26.0,16.7,0,10.0,21.1,5.0
8240,2023-12-10 08:00,24.9,1,16.6,83,1017.0,0.00,60,41,26.2,24.9,26.2,16.5,0,10.0,21.2,6.0
8241,2023-12-10 09:00,25.4,1,17.3,88,1017.0,0.01,59,53,26.5,25.4,26.5,16.7,0,10.0,20.4,6.0


In [ ]:
df = data_scaler(df)  #normalise the data
df_plot = long_form(df[0:200])

In [ ]:
altair_plot(df_plot)

alt.LayerChart(...)

## Forecast

In [ ]:
url = Weather_api_url(token).forecast(10)
data_json = weather_request(url)
df = data_filter(data_json, 'forecast')
df = data_scaler(df)

In [ ]:
import pandas as pd
df.shape

In [ ]:
df_plot = long_form(df)
altair_plot(df_plot)